In [ ]:
#!/bin/bash

cd ~

bash Linux-install-scripts/RaspberryPi/pushbullet

In [ ]:
echo "Installing python, pip, Jupyter and related other packages"
sudo apt update -qq
sudo apt upgrade -qq

In [5]:
sudo curl -sL https://deb.nodesource.com/setup_15.x | sudo bash -

tput: unknown terminal "unknown"

## Installing the NodeSource Node.js 15.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 http://raspbian.raspberrypi.org/raspbian buster InRelease [15.0 kB]
Hit:2 http://archive.raspberrypi.org/debian buster InRelease                   
Hit:3 https://deb.nodesource.com/node_15.x buster InRelease                    
Fetched 15.0 kB in 3s (5,964 B/s)
Reading package lists... Done

## Confirming "buster" is supported...

+ curl -sLf -o /dev/null 'https://deb.nodesource.com/node_15.x/dists/buster/Release'

## Adding the NodeSource signing key to your keyring...

+ curl -s https://deb.nodesource.com/gpgkey/nodesource.gpg.key | apt-key add -
OK

## Creating apt sources list file for the NodeSource Node.js 15.x repo...

+ echo 'deb https://deb.nodesource.com/node_15.x buster main' > /etc/apt/sources.list.d/nodesource.list
+ echo 'deb-src https://deb.nodesource.com/node_15.x buster main' >> /etc/apt/sources.list.d/nodesource.list

## Running 

In [ ]:
sudo apt install -q python3 python3-pip python3-venv libffi-dev -y
sudo pip3 install -q setuptools cffi
pip3 install -q jupyterlab ipywidgets bokeh ipyleaflet atoti voila

Installing python, pip, Jupyter and related other packages


In [ ]:
if [[ $(sudo grep -Ril "\[Unit\]" /etc/systemd/system/jupyter.service) == "/etc/systemd/system/jupyter.service" ]]; then
	echo "service file exists: no file created"
else
	sudo tee -a /etc/systemd/system/jupyter.service > /dev/null <<EOT
[Unit]
Description=Jupyter Lab

[Service]
Type=simple
PIDFile=/run/jupyter.pid
ExecStart=/bin/bash -c "~/.local/bin/jupyter-lab"
User=$USER
Group=pi
WorkingDirectory=~
Restart=always
RestartSec=10

[Install]
WantedBy=multi-user.target
EOT
fi
sudo systemctl enable jupyter.service
sudo systemctl daemon-reload

In [ ]:
mkdir ~/.certificates
cd ~/.certificates
openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout jupyter.key -out jupyter.pem

openssl req -new -nodes -out server.csr -newkey rsa:2048 -keyout server.key
sudo tee -a /etc/systemd/system/jupyter.service > /dev/null <<EOT
authorityKeyIdentifier=keyid,issuer
basicConstraints=CA:FALSE
keyUsage = digitalSignature, nonRepudiation, keyEncipherment, dataEncipherment
subjectAltName = @alt_names

[alt_names]
DNS.1 = example.dom
DNS.2 = ex-ample.dom" >> v3.ext
openssl x509 -req -in server.csr -CA jupyter.pem -CAkey jupyter.key -CAcreateserial -out server.crt -days 500 -sha256 -extfile v3.ext
openssl pkcs12 -inkey server.key -in server.crt -export -out server.pfx

In [ ]:
cd ~
jupyter notebook --generate-config
jupyter notebook password

echo "Set up https with instructions on https://jupyter-notebook.readthedocs.io/en/stable/public_server.html#using-ssl-for-encrypted-communication"